# Topic 5. Recursive functions and data types

The goals of this topic are to understand:

* How recursive types (lists, trees, etc.) are defined algebraically
* How functions over recursive types are defined recursivelly
* The two major types of recursive functions: general and tail-recursive

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 16. Working with Lists
- Chapter 26. Extractors (optional)

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapters 29-36. Recursion.

## Recursive types

### The `List` type

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the following algebraic equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



The implementation in Scala is similar to the following one (we also give the generic version `List[A]`, rather than the implementation of `IntList`):

In [1]:
object StdDefinition:
    enum List[A]:
        case NonEmpty(head: A, tail: List[A])
        case Empty()

defined object StdDefinition

However the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. This forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times.  The standard definition looks like as follows:

In [2]:
object ActualStdDefinition:
    enum List[+A]:
        case ::(head: A, tail: List[A])
        case Nil

defined object ActualStdDefinition

### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [3]:
// Less beautifully 
val l1: List[Int] = ::(1, ::(2, ::(3, Nil)))

// More idiomatically
val l2: List[Int] = 1 :: 2 :: 3 :: Nil
val l3: List[Int] = List(1,2,3)

l1: List[Int] = List(1, 2, 3)
l2: List[Int] = List(1, 2, 3)
l3: List[Int] = List(1, 2, 3)

And we can also pattern match on lists, similarly:

In [4]:
// Less beautifully
l1 match
    case Nil => 0
    case ::(head, tail) => 1

// more idiomatically
l1 match
    case Nil => 0
    case head :: tail => 1

// or
l1 match
    case List() => 0
    case List(h1, h2, h3) => 1
    case _ => 2

res4_0: Int = 1
res4_1: Int = 1
res4_2: Int = 1

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [5]:
// Using mutable variables

def lengthI[A](list: List[A]): Int =
    var out: Int = 0
    var aux: List[A] = list
    while (aux != Nil)
        aux = aux.tail
        out += 1
    out

defined function lengthI

In [6]:
lengthI(List())
lengthI(List(1,2,3,4))

res6_0: Int = 0
res6_1: Int = 4

The recursive function is implemented as follows: 

In [7]:
// Using recursive functions

def lengthR[A](list: List[A]): Int = 
    list match
        case Nil           => 0
        case _ :: tail => 1 + lengthR(tail)

defined function lengthR

Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that are available, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with very long lists.

### Tail-recursive functions

The implementation using tail-recursion solves the issues with the stack. It commonly makes use of auxiliary functions:

In [8]:
// Using tail-recursive functions

def lengthTR[A](list: List[A]): Int =

    @annotation.tailrec
    def lengthAux(out: Int, aux: List[A]): Int = 
        aux match
            case Nil => out
            case _ :: tail => lengthAux(out+1, tail)
    
    lengthAux(0, list)

defined function lengthTR

In [9]:
lengthTR(List())
lengthTR(List(1,2,3))

res9_0: Int = 0
res9_1: Int = 3

We can check the stack-safety problems of non-tail recursive functions by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [10]:
// First, imperatively

def constantList[A](value: A, length: Int): List[A] =
    var out: List[A] = Nil
    for (i <- 1 to length)
        out = value :: out
    out

defined function constantList

In [11]:
// Next, tail-recursively

def constantList[A](value: A, length: Int): List[A] =

    def constantAux(out: List[A], i: Int): List[A] = 
        if (i == 0) out
        else constantAux(value :: out, i-1)
    
    constantAux(Nil, length)

defined function constantList

We can also use the function [`fill`](https://www.scala-lang.org/api/2.13.3/scala/collection/immutable/List$.html#fill[A](n:Int)(elem:=%3EA):CC[A]) of the Scala standard library.

Now, let's calculate the length of a list long enough to blow up the stack, using each of the three implementations:

In [12]:
// Imperatively
lengthI(constantList(0, 100000))

res12: Int = 100000

In [13]:
// Tail-recursive
lengthTR(constantList(0, 100000))

res13: Int = 100000

In [13]:
// Plain recursive
// lengthR(constantList(0, 10000))

### Unit testing with `scalatest`

In [14]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. The test catalogue receives the actual function to be tested as a parameter. For instance, this is a possible test class for the `length` function:

In [15]:
class TestLength(lengthF: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "length" should "work" in:
        lengthF(List()) shouldBe 0 
        lengthF(List(1)) shouldBe 1 
        lengthF(List(1,2,3,4)) shouldBe 4

defined class TestLength

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

In [16]:
run(TestLength(lengthR))

cell15$Helper$TestLength:
length
- should work


### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [17]:
class TestSum(sum: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "sum" should "work" in:
        sum(List()) shouldBe 0 
        sum(List(1)) shouldBe 1 
        sum(List(1,2,3,4)) shouldBe 10

defined class TestSum

In [18]:
// Recursively

def sum(list: List[Int]): Int = 
    list match
        case Nil => 0 : Int
        case head :: tail => head + sum(tail) : Int 

defined function sum

In [19]:
run(TestSum(sum))

cell17$Helper$TestSum:
sum
- should work


In [20]:
// With tail-recursion

def sumTR(list: List[Int]): Int =

    def sumAux(acc: Int, list: List[Int]): Int = 
        list match
            case Nil => acc : Int
            case head :: tail => sumAux(head + acc, tail) : Int 
    
    sumAux(0, list)

defined function sumTR

In [21]:
run(TestSum(sumTR))

cell17$Helper$TestSum:
sum
- should work


### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [22]:
class TestProduct(product: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "product" should "work" in:
        product(List()) shouldBe 1 
        product(List(1)) shouldBe 1 
        product(List(1,2,3,4)) shouldBe 24
        product(List(1,0,5,6,7)) shouldBe 0

defined class TestProduct

 This is the common recursive implementation:

In [23]:
def product(list: List[Int]): Int = 
    list match
        case Nil => 1
        case head :: tail => 
            head * product(tail)

defined function product

In [24]:
run(TestProduct(product))

cell22$Helper$TestProduct:
product
- should work


But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [25]:
def product2(list: List[Int]): Int = 
    list match
        case Nil => 1
        // We add this extra case
        case 0 :: _ => 0
        case head :: tail => head * product2(tail)

defined function product2

In [26]:
run(TestProduct(product2))

cell22$Helper$TestProduct:
product
- should work


A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [27]:
class TestMember(member: (List[Int], Int) => Boolean) extends AnyFlatSpec with should.Matchers:
    "member" should "work" in:
        member(List(), 6) shouldBe false
        member(List(1), 1) shouldBe true
        member(List(1), 3) shouldBe false
        member(List(1,2,3,4), 4) shouldBe true

defined class TestMember

In [28]:
def member[A](list: List[A], elem: A): Boolean = 
    list match
        case Nil => false
        case head :: tail => 
            head == elem || member(tail, elem)

defined function member

In [29]:
run(TestMember(member))

cell27$Helper$TestMember:
member
- should work


We can also pattern match against a specific value as follows:

In [30]:
def member[A](list: List[A], elem: A): Boolean = 
    list match
        case Nil => false
        case `elem` :: _ => true
        case _ :: tail => member(tail, elem)

defined function member

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [31]:
class TestLast(last: List[Int] => Option[Int]) extends AnyFlatSpec with should.Matchers:
    "last" should "work" in:
        last(List()) shouldBe None
        last(List(1)) shouldBe Some(1)
        last(List(1,2,3,4)) shouldBe Some(4)

defined class TestLast

In [32]:
@annotation.tailrec
def last[A](list: List[A]): Option[A] =
    list match
        case Nil => None
        case head :: Nil => Some(head)
        case head :: tail => last(tail)

defined function last

In [33]:
run(TestLast(last))

cell31$Helper$TestLast:
last
- should work


### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [47]:
class TestInsertLast(insertLast: (List[Int], Int) => List[Int]) 
extends AnyFlatSpec with should.Matchers:
    "insertLast" should "work" in:
        insertLast(List(), 6) shouldBe List(6)
        insertLast(List(1), 1) shouldBe List(1,1)
        insertLast(List(1,2,3,4), 4) shouldBe List(1,2,3,4,4)

defined class TestInsertLast

In [48]:
def insertLast[A](list: List[A], elem: A): List[A] = 
    list match
        case Nil => List(elem)
        case head :: tail => 
            head :: insertLast(tail, elem)

defined function insertLast

In [49]:
run(TestInsertLast(insertLast))

cell47$Helper$TestInsertLast:
insertLast
- should work


### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [50]:
class TestReverse(reverse: List[Int] => List[Int]) extends AnyFlatSpec with should.Matchers:
    "reverse" should "work" in:
        reverse(List()) shouldBe List()
        reverse(List(1)) shouldBe List(1)
        reverse(List(1,2,3)) shouldBe List(3,2,1)

defined class TestReverse

In [51]:
// Really inefficient 

def reverse[A](list: List[A]): List[A] = 
    list match
        case Nil => Nil
        case head :: tail =>
                insertLast(reverse(tail), head)

defined function reverse

In [52]:
run(TestReverse(reverse))

cell50$Helper$TestReverse:
reverse
- should work


In [53]:
// Tail-recursive, efficiently

def reverseTR[A](list: List[A]): List[A] =
    def reverseAux(acc: List[A], list: List[A]): List[A] = 
        list match
            case Nil => acc
            case head :: tail => 
                reverseAux(head :: acc, tail)
    
    reverseAux(Nil, list)

defined function reverseTR

In [54]:
run(TestReverse(reverseTR))

cell50$Helper$TestReverse:
reverse
- should work


### Example: concatenate lists

In [56]:
class TestConcatenate(concatenate: (List[Int], List[Int]) => List[Int]) 
extends AnyFlatSpec with should.Matchers:
    "concatenate" should "work" in:
        concatenate(List(), List()) shouldBe List()
        concatenate(List(1), List()) shouldBe List(1)
        concatenate(List(), List(1)) shouldBe List(1)
        concatenate(List(1,2,3), List(1,3)) shouldBe List(1,2,3,1,3)
        concatenate(List(1,2,3), List(4,5,6)) shouldBe List(1,2,3,4,5,6)

defined class TestConcatenate

In [57]:
def concatenate[A](list1: List[A], list2: List[A]): List[A] =
    list1 match
        case Nil => 
            list2 : List[A]
        case head :: tail => 
            (head :: concatenate(tail, list2)) : List[A]

defined function concatenate

In [58]:
run(TestConcatenate(concatenate))

cell56$Helper$TestConcatenate:
concatenate
- should work


Tail-recursive concatenation:

In [59]:
def concatenateTR[A](list1: List[A], list2: List[A]): List[A] =

    def concAux(acc: List[A], list: List[A]): List[A] = 
        list match
            case Nil => acc
            case head :: tail => 
                concAux(head :: acc, tail)
    
    concAux(list2, concAux(Nil, list1))

defined function concatenateTR

In [60]:
run(TestConcatenate(concatenateTR))

cell56$Helper$TestConcatenate:
concatenate
- should work
